### Import necessary libraryies

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 122kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 710kB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=60a9331215a6acbc6ab919c6c7a5abdca555ba92651f9923ed34c0d2c8b3fc1e
  Stored in directory: /Users/justinwu/Library/Caches/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Folium installed
Libraries imported.


### Foursqure setting

In [39]:
CLIENT_ID = 'FGF1SK2HYOHKUIPSVAKXSEPTVYRRS3FMS4OE4WTWKDDYSHTE' # your Foursquare ID
CLIENT_SECRET = '1FDO4E41CFGZBFYRKGE1TOXR2NM5DD0MIFNCC2GY5RHH5SS4' # your Foursquare Secret
ACCESS_TOKEN = 'UYPUGMAYB2TFJACUJN3BAM5ERGZZUHW3B1NQTH0LL12K2RN2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 3000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FGF1SK2HYOHKUIPSVAKXSEPTVYRRS3FMS4OE4WTWKDDYSHTE
CLIENT_SECRET:1FDO4E41CFGZBFYRKGE1TOXR2NM5DD0MIFNCC2GY5RHH5SS4


In [86]:
Taipei_address = "Taipei city"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(Taipei_address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.0452747 121.5078001


In [109]:
search_query = 'school'
radius = 30000
print(search_query + ' .... OK!')

school .... OK!


In [110]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [186]:
results = requests.get(url).json()

In [187]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.country,location.formattedAddress,location.address,location.postalCode,location.state,location.neighborhood
0,4f7d8d7ee4b02c4143c1c4b4,Maiden School 夢幻學園,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1628395625,False,臺北地下街 Y27付近,25.050124,121.511053,"[{'label': 'display', 'lat': 25.05012382388512...",631,TW,台北市,臺灣,[台北市],NaN,NaN,NaN,NaN
1,4cc56b843d7fa1cd5263b15f,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,"[{'id': '4f4533814b9074f6e4fb0106', 'name': 'M...",v-1628395625,False,NaN,25.051248,121.507985,"[{'label': 'display', 'lat': 25.05124849594919...",665,TW,大同區,臺灣,"[臺北市西寧北路32號, 大同區, 臺北市 10341]",臺北市西寧北路32號,10341,臺北市,NaN
2,4fe7b2f06d86e50fb8cb2255,ChengYuan High School (成淵高級中學),"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",v-1628395625,False,NaN,25.061129,121.519132,"[{'label': 'display', 'lat': 25.06112944194507...",2102,TW,Datong District,臺灣,"[台北市承德路二段235號, Ta-t’ung, 臺北市 10364]",台北市承德路二段235號,10364,臺北市,NaN
3,4c8445f6d34ca143029f3d80,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,"[{'id': '4f4533804b9074f6e4fb0105', 'name': 'E...",v-1628395625,False,NaN,25.037744,121.520599,"[{'label': 'display', 'lat': 25.03774373596082...",1539,TW,中正區,臺灣,"[臺北市仁愛路一段2~4號, 中正區, 臺北市 100]",臺北市仁愛路一段2~4號,100,臺北市,NaN
4,4d0b25c2d8d78cfadf271bbe,臺北市立第一女子高級中學 Taipei First Girls High School,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",v-1628395625,False,NaN,25.038775,121.512982,"[{'label': 'display', 'lat': 25.03877451047955...",892,TW,中正區,臺灣,"[臺北市重慶南路一段 165 號, 中正區, 臺北市 10045]",臺北市重慶南路一段 165 號,10045,臺北市,NaN


In [192]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
print(dataframe_filtered['categories'].value_counts())
Taipei_school = dataframe_filtered[dataframe_filtered['categories'].isin(['Elementary School', 'High School', 'Middle School'])]


Taipei_school.head()

Elementary School                  18
Middle School                      13
High School                        10
Metro Station                       3
School                              2
Salon / Barbershop                  1
Student Center                      1
Café                                1
College Administrative Building     1
Name: categories, dtype: int64


,name,categories,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,country,formattedAddress,address,postalCode,state,neighborhood,id
1,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,Middle School,NaN,25.051248,121.507985,"[{'label': 'display', 'lat': 25.05124849594919...",665,TW,大同區,臺灣,"[臺北市西寧北路32號, 大同區, 臺北市 10341]",臺北市西寧北路32號,10341,臺北市,NaN,4cc56b843d7fa1cd5263b15f
2,ChengYuan High School (成淵高級中學),High School,NaN,25.061129,121.519132,"[{'label': 'display', 'lat': 25.06112944194507...",2102,TW,Datong District,臺灣,"[台北市承德路二段235號, Ta-t’ung, 臺北市 10364]",台北市承德路二段235號,10364,臺北市,NaN,4fe7b2f06d86e50fb8cb2255
3,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,Elementary School,NaN,25.037744,121.520599,"[{'label': 'display', 'lat': 25.03774373596082...",1539,TW,中正區,臺灣,"[臺北市仁愛路一段2~4號, 中正區, 臺北市 100]",臺北市仁愛路一段2~4號,100,臺北市,NaN,4c8445f6d34ca143029f3d80
4,臺北市立第一女子高級中學 Taipei First Girls High School,High School,NaN,25.038775,121.512982,"[{'label': 'display', 'lat': 25.03877451047955...",892,TW,中正區,臺灣,"[臺北市重慶南路一段 165 號, 中正區, 臺北市 10045]",臺北市重慶南路一段 165 號,10045,臺北市,NaN,4d0b25c2d8d78cfadf271bbe
5,臺北市立建成國民中學 Taipei Municipal JianCheng Junior H...,Middle School,NaN,25.051614,121.519529,"[{'label': 'display', 'lat': 25.05161438248922...",1377,TW,大同區,臺灣,"[台北市長安西路37-1號, 大同區, 臺北市 10351]",台北市長安西路37-1號,10351,臺北市,NaN,4ee89ebe93ad8b68e9a17966


In [195]:
Taipei_school_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(Taipei_school.lat, Taipei_school.lng, Taipei_school.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Taipei_school_map
            )
    
    
Taipei_school_map

In [218]:
column_selected = ['name', 'lat', 'lng', 'categories']
Taipei_school = Taipei_school.loc[:,column_selected].reset_index(drop=True)
Taipei_school.head()

,name,lat,lng,categories
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,Middle School
1,ChengYuan High School (成淵高級中學),25.061129,121.519132,High School
2,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,25.037744,121.520599,Elementary School
3,臺北市立第一女子高級中學 Taipei First Girls High School,25.038775,121.512982,High School
4,臺北市立建成國民中學 Taipei Municipal JianCheng Junior H...,25.051614,121.519529,Middle School


In [125]:
# Search for venues with high schools' locations

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [199]:
Taipei_school_venue = getNearbyVenues(names = Taipei_school['name'],
                                   latitudes=Taipei_school['lat'],
                                   longitudes=Taipei_school['lng'])

臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior High School
ChengYuan High School (成淵高級中學)
臺北市立東門國民小學 Taipei Municipal DongMen Elementary School
臺北市立第一女子高級中學 Taipei First Girls High School
臺北市立建成國民中學 Taipei Municipal JianCheng Junior High School
臺北市立南門國民中學 Taipei Municipal NanMen Junior High School
臺北市立弘道國民中學 Taipei Municipal HongDao Junior High School
臺北市立成功高級中學 Taipei Municipal ChengGong High School
臺北市立福星國民小學 Taipei Municipal FuXing Elementary School
臺北市立建國高級中學 Taipei Municipal JianGuo High School
臺北市立中正國民中學 Taipei Municipal ZhongZheng Junior High School
台北市立萬華國民中學 Taipei Municipal WanHua Junior High School
臺北市立民權國民中學 Taipei Municipal MingQuan Junior High School
臺北市立大橋國民小學 Taipei Municipal Dachiao Elementary School
臺北市立蘭州國民中學 Taipei Municipal LanZhou Junior High School
臺北市立中山女子高級中學 Taipei Municipal Zhongshan Girls' High School
台北市立古亭國民中學 Taipei Municipal GuTing Junior High School
臺北市立幸安國民小學 Taipei Municipal XingAn Elementary School
臺北市立長安國民中學 Taipei Municipal ChangAn Junior High School

In [200]:
print(Taipei_school_venue.shape)
Taipei_school_venue

(1426, 7)


,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,Mikkeller Bar Taipei (米凱樂台灣),25.053728,121.510364,Beer Bar
1,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,永樂担仔麵,25.054496,121.510398,Taiwanese Restaurant
2,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,金仙魚丸分店,25.054547,121.510330,Taiwanese Restaurant
3,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,Fuhong Beef Noodles (富宏牛肉麵),25.047382,121.507600,Noodle House
4,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,A Design&Life Project,25.053364,121.509873,Furniture / Home Store
...,...,...,...,...,...,...,...
1421,新北市永平國民小學 New Taipei Municipal YongPin Element...,25.007648,121.504567,日日村 A Day,25.009690,121.505978,Japanese Restaurant
1422,新北市永平國民小學 New Taipei Municipal YongPin Element...,25.007648,121.504567,老虎醬 溫州大餛飩,25.007777,121.507735,Chinese Restaurant
1423,新北市永平國民小學 New Taipei Municipal YongPin Element...,25.007648,121.504567,世界宗教博物館 World Religions Museum,25.008011,121.508398,Museum
1424,新北市永平國民小學 New Taipei Municipal YongPin Element...,25.007648,121.504567,FamilyMart (全家),25.009278,121.500016,Convenience Store


In [201]:
Taipei_school_venue.groupby('School').count()

,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
School,,,,,,
ChengYuan High School (成淵高級中學),20,20,20,20,20,20
Taipei Municipal ChangChun Elementary School (臺北市立長春國民小學),75,75,75,75,75,75
Taipei Municipal HuLu Elementary School (臺北市葫蘆國民小學),4,4,4,4,4,4
Taipei Municipal JiLin Elementary School (臺北市立吉林國民小學),90,90,90,90,90,90
Taipei Municipal ZhongShan Elementary School (臺北市中山國民小學),74,74,74,74,74,74
台北市私立強恕高級中學 Taipei Qiang Shu Private High School,46,46,46,46,46,46
台北市立古亭國民中學 Taipei Municipal GuTing Junior High School,6,6,6,6,6,6
台北市立民權國民小學 Taipei Municipal MinQuan Elementary School,30,30,30,30,30,30
台北市立民生國民小學 Taipei Municipal MinSheng Elementary School,35,35,35,35,35,35


In [202]:
# one hot encoding 
Taipei_school_onehot = pd.get_dummies(Taipei_school_venue[['Venue Category']], prefix="", prefix_sep="")

In [203]:
Taipei_school_onehot['School'] = Taipei_school_venue['School']

In [204]:
Fixed_column = [Taipei_school_onehot.columns[-1]] + list(Taipei_school_onehot.columns[:-1])
Taipei_school_onehot = Taipei_school_onehot[Fixed_column]

Taipei_school_onehot

,School,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,BBQ Joint,...,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,新北市永平國民小學 New Taipei Municipal YongPin Element...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1422,新北市永平國民小學 New Taipei Municipal YongPin Element...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1423,新北市永平國民小學 New Taipei Municipal YongPin Element...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1424,新北市永平國民小學 New Taipei Municipal YongPin Element...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
Taipei_school_onehot_grouped = Taipei_school_onehot.groupby('School').mean().reset_index()

In [207]:
Taipei_school_onehot_grouped.head()

,School,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,BBQ Joint,...,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant
0,ChengYuan High School (成淵高級中學),0.000000,0.0,0.0,0.0,0.000000,0.050000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,Taipei Municipal ChangChun Elementary School (...,0.013333,0.0,0.0,0.0,0.013333,0.053333,0.0,0.0,0.013333,...,0.000000,0.0,0.013333,0.026667,0.0,0.0,0.0,0.0,0.0,0.000000
2,Taipei Municipal HuLu Elementary School (臺北市葫蘆...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,Taipei Municipal JiLin Elementary School (臺北市立...,0.000000,0.0,0.0,0.0,0.011111,0.055556,0.0,0.0,0.022222,...,0.011111,0.0,0.011111,0.044444,0.0,0.0,0.0,0.0,0.0,0.000000
4,Taipei Municipal ZhongShan Elementary School (...,0.013514,0.0,0.0,0.0,0.000000,0.013514,0.0,0.0,0.013514,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.013514


In [208]:
num_top_venues = 5

for hood in Taipei_school_onehot_grouped['School']:
    print("----"+hood+"----")
    temp = Taipei_school_onehot_grouped[Taipei_school_onehot_grouped['School'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ChengYuan High School (成淵高級中學)----
                  venue  freq
0          Dessert Shop   0.1
1           Coffee Shop   0.1
2  Taiwanese Restaurant   0.1
3   Japanese Restaurant   0.1
4     Convenience Store   0.1


----Taipei Municipal ChangChun Elementary School (臺北市立長春國民小學)----
                  venue  freq
0     Convenience Store  0.17
1    Chinese Restaurant  0.11
2                 Hotel  0.08
3  Taiwanese Restaurant  0.07
4      Asian Restaurant  0.05


----Taipei Municipal HuLu Elementary School (臺北市葫蘆國民小學)----
                 venue  freq
0          Supermarket  0.25
1         Burger Joint  0.25
2  Dumpling Restaurant  0.25
3         Optical Shop  0.25
4          Pastry Shop  0.00


----Taipei Municipal JiLin Elementary School (臺北市立吉林國民小學)----
                  venue  freq
0     Convenience Store  0.13
1  Taiwanese Restaurant  0.10
2    Chinese Restaurant  0.08
3     Hotpot Restaurant  0.06
4      Asian Restaurant  0.06


----Taipei Municipal ZhongShan Elementary School (臺

                  venue  freq
0    Chinese Restaurant  0.14
1  Taiwanese Restaurant  0.09
2          Night Market  0.09
3           Coffee Shop  0.05
4                 Hotel  0.05


----臺北市立蘭州國民中學 Taipei Municipal LanZhou Junior High School----
               venue  freq
0  Convenience Store  0.15
1               Café  0.15
2       Noodle House  0.07
3   Asian Restaurant  0.07
4        Coffee Shop  0.07


----臺北市立長安國民中學 Taipei Municipal ChangAn Junior High School----
                  venue  freq
0   Japanese Restaurant  0.16
1                  Café  0.10
2  Taiwanese Restaurant  0.10
3     Hotpot Restaurant  0.06
4      Sushi Restaurant  0.06


----臺北市立龍門國民中學 Taipei Municipal LongMen Junior High School----
               venue  freq
0               Café  0.16
1        Coffee Shop  0.06
2           Tea Room  0.06
3  Convenience Store  0.06
4   Sushi Restaurant  0.04




In [148]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [209]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
school_venues_sorted = pd.DataFrame(columns=columns)
school_venues_sorted['School'] = Taipei_school_onehot_grouped['School']

for ind in np.arange(Taipei_school_onehot_grouped.shape[0]):
    school_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Taipei_school_onehot_grouped.iloc[ind, :], num_top_venues)

school_venues_sorted.head()

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ChengYuan High School (成淵高級中學),Dessert Shop,Japanese Restaurant,Taiwanese Restaurant,Coffee Shop,Convenience Store,Dumpling Restaurant,Laser Tag,Fast Food Restaurant,Market,Soup Place
1,Taipei Municipal ChangChun Elementary School (...,Convenience Store,Chinese Restaurant,Hotel,Taiwanese Restaurant,Asian Restaurant,Café,Japanese Restaurant,Hotpot Restaurant,Fast Food Restaurant,Noodle House
2,Taipei Municipal HuLu Elementary School (臺北市葫蘆...,Optical Shop,Supermarket,Burger Joint,Dumpling Restaurant,Yunnan Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flea Market
3,Taipei Municipal JiLin Elementary School (臺北市立...,Convenience Store,Taiwanese Restaurant,Chinese Restaurant,Hotpot Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Hotel,Coffee Shop,Japanese Restaurant
4,Taipei Municipal ZhongShan Elementary School (...,Japanese Restaurant,Convenience Store,Chinese Restaurant,Taiwanese Restaurant,Hotel,Fast Food Restaurant,Hotpot Restaurant,Bar,Café,Supermarket


## Cluster Analysis

In [210]:
# set number of clusters
kclusters = 3

Taipei_school_clusering = Taipei_school_onehot_grouped.drop('School', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Taipei_school_clusering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 1, 0, 1, 1, 0], dtype=int32)

In [211]:
school_venues_sorted.insert(0, 'Clusterlabel', kmeans.labels_)


In [212]:
school_venues_sorted

,Clusterlabel,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,ChengYuan High School (成淵高級中學),Dessert Shop,Japanese Restaurant,Taiwanese Restaurant,Coffee Shop,Convenience Store,Dumpling Restaurant,Laser Tag,Fast Food Restaurant,Market,Soup Place
1,0,Taipei Municipal ChangChun Elementary School (...,Convenience Store,Chinese Restaurant,Hotel,Taiwanese Restaurant,Asian Restaurant,Café,Japanese Restaurant,Hotpot Restaurant,Fast Food Restaurant,Noodle House
2,1,Taipei Municipal HuLu Elementary School (臺北市葫蘆...,Optical Shop,Supermarket,Burger Joint,Dumpling Restaurant,Yunnan Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flea Market
3,0,Taipei Municipal JiLin Elementary School (臺北市立...,Convenience Store,Taiwanese Restaurant,Chinese Restaurant,Hotpot Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Hotel,Coffee Shop,Japanese Restaurant
4,0,Taipei Municipal ZhongShan Elementary School (...,Japanese Restaurant,Convenience Store,Chinese Restaurant,Taiwanese Restaurant,Hotel,Fast Food Restaurant,Hotpot Restaurant,Bar,Café,Supermarket
5,1,台北市私立強恕高級中學 Taipei Qiang Shu Private High School,Café,Japanese Restaurant,Convenience Store,Chinese Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Taiwanese Restaurant,Park,Noodle House,Dumpling Restaurant
6,0,台北市立古亭國民中學 Taipei Municipal GuTing Junior High...,Convenience Store,Dessert Shop,Japanese Restaurant,Food Truck,Market,Fast Food Restaurant,French Restaurant,Food Stand,Food Court,Food & Drink Shop
7,1,台北市立民權國民小學 Taipei Municipal MinQuan Elementary...,Café,Convenience Store,Noodle House,Taiwanese Restaurant,Italian Restaurant,Park,Chinese Restaurant,Hotpot Restaurant,Dessert Shop,Dumpling Restaurant
8,1,台北市立民生國民小學 Taipei Municipal MinSheng Elementar...,Café,Chinese Restaurant,Dumpling Restaurant,Pastry Shop,Hotel,Taiwanese Restaurant,French Restaurant,Convenience Store,Comfort Food Restaurant,Shabu-Shabu Restaurant
9,0,台北市立萬華國民中學 Taipei Municipal WanHua Junior High...,Convenience Store,Coffee Shop,Supermarket,Hotel,Bakery,Chinese Restaurant,Yunnan Restaurant,Fast Food Restaurant,Food Truck,Food Stand


In [219]:
Taipei_school_merge = Taipei_school[column_selected]
Taipei_school_merge.columns = ['School', 'latitude', 'longitude', 'Categories']
Taipei_school_merge.head()

,School,latitude,longitude,Categories
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,Middle School
1,ChengYuan High School (成淵高級中學),25.061129,121.519132,High School
2,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,25.037744,121.520599,Elementary School
3,臺北市立第一女子高級中學 Taipei First Girls High School,25.038775,121.512982,High School
4,臺北市立建成國民中學 Taipei Municipal JianCheng Junior H...,25.051614,121.519529,Middle School


In [220]:
Taipei_school_merge = Taipei_school_merge.join(school_venues_sorted.set_index('School'), on='School')

Taipei_school_merge.head()

,School,latitude,longitude,Categories,Clusterlabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,25.051248,121.507985,Middle School,0,Noodle House,Café,Taiwanese Restaurant,Hotel,Convenience Store,Gym / Fitness Center,Coffee Shop,Furniture / Home Store,Fabric Shop,Dessert Shop
1,ChengYuan High School (成淵高級中學),25.061129,121.519132,High School,0,Dessert Shop,Japanese Restaurant,Taiwanese Restaurant,Coffee Shop,Convenience Store,Dumpling Restaurant,Laser Tag,Fast Food Restaurant,Market,Soup Place
2,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,25.037744,121.520599,Elementary School,1,Food Court,Theater,Monument / Landmark,Concert Hall,Café,Breakfast Spot,Dumpling Restaurant,Sculpture Garden,Tea Room,Juice Bar
3,臺北市立第一女子高級中學 Taipei First Girls High School,25.038775,121.512982,High School,0,Chinese Restaurant,Convenience Store,Park,Noodle House,Café,Hostel,Museum,Bike Rental / Bike Share,Coffee Shop,Bubble Tea Shop
4,臺北市立建成國民中學 Taipei Municipal JianCheng Junior H...,25.051614,121.519529,Middle School,1,Coffee Shop,Hotel,Ramen Restaurant,Train Station,Chinese Restaurant,Department Store,Bakery,Japanese Restaurant,Gourmet Shop,Gym


In [215]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Taipei_school_merge['latitude'], Taipei_school_merge['longitude'], Taipei_school_merge['School'], Taipei_school_merge['Clusterlabel']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1 :

In [225]:
Taipei_school_merge.loc[Taipei_school_merge['Clusterlabel'] == 0, Taipei_school_merge.columns[[0,3] + list(range(4, Taipei_school_merge.shape[1]))]]

,School,Categories,Clusterlabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,臺北市立忠孝國民中學 Taipei Municipal ZhongXiao Junior H...,Middle School,0,Noodle House,Café,Taiwanese Restaurant,Hotel,Convenience Store,Gym / Fitness Center,Coffee Shop,Furniture / Home Store,Fabric Shop,Dessert Shop
1,ChengYuan High School (成淵高級中學),High School,0,Dessert Shop,Japanese Restaurant,Taiwanese Restaurant,Coffee Shop,Convenience Store,Dumpling Restaurant,Laser Tag,Fast Food Restaurant,Market,Soup Place
3,臺北市立第一女子高級中學 Taipei First Girls High School,High School,0,Chinese Restaurant,Convenience Store,Park,Noodle House,Café,Hostel,Museum,Bike Rental / Bike Share,Coffee Shop,Bubble Tea Shop
11,台北市立萬華國民中學 Taipei Municipal WanHua Junior High...,Middle School,0,Convenience Store,Coffee Shop,Supermarket,Hotel,Bakery,Chinese Restaurant,Yunnan Restaurant,Fast Food Restaurant,Food Truck,Food Stand
14,臺北市立蘭州國民中學 Taipei Municipal LanZhou Junior Hig...,Middle School,0,Convenience Store,Café,Asian Restaurant,Taiwanese Restaurant,Noodle House,Coffee Shop,Hotel,Japanese Restaurant,Gym / Fitness Center,Metro Station
15,臺北市立中山女子高級中學 Taipei Municipal Zhongshan Girls'...,High School,0,Convenience Store,Chinese Restaurant,Hotel,Japanese Restaurant,Café,Hotpot Restaurant,Coffee Shop,Dessert Shop,Noodle House,Restaurant
16,台北市立古亭國民中學 Taipei Municipal GuTing Junior High...,Middle School,0,Convenience Store,Dessert Shop,Japanese Restaurant,Food Truck,Market,Fast Food Restaurant,French Restaurant,Food Stand,Food Court,Food & Drink Shop
17,臺北市立幸安國民小學 Taipei Municipal XingAn Elementary ...,Elementary School,0,Chinese Restaurant,Hotel,Café,Dumpling Restaurant,Flea Market,Asian Restaurant,Convenience Store,Metro Station,Coffee Shop,Noodle House
20,Taipei Municipal ZhongShan Elementary School (...,Elementary School,0,Japanese Restaurant,Convenience Store,Chinese Restaurant,Taiwanese Restaurant,Hotel,Fast Food Restaurant,Hotpot Restaurant,Bar,Café,Supermarket
21,Taipei Municipal JiLin Elementary School (臺北市立...,Elementary School,0,Convenience Store,Taiwanese Restaurant,Chinese Restaurant,Hotpot Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Hotel,Coffee Shop,Japanese Restaurant


### Cluster2 :

In [226]:
Taipei_school_merge.loc[Taipei_school_merge['Clusterlabel'] == 1, Taipei_school_merge.columns[[0,3] + list(range(4, Taipei_school_merge.shape[1]))]]

,School,Categories,Clusterlabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,臺北市立東門國民小學 Taipei Municipal DongMen Elementary...,Elementary School,1,Food Court,Theater,Monument / Landmark,Concert Hall,Café,Breakfast Spot,Dumpling Restaurant,Sculpture Garden,Tea Room,Juice Bar
4,臺北市立建成國民中學 Taipei Municipal JianCheng Junior H...,Middle School,1,Coffee Shop,Hotel,Ramen Restaurant,Train Station,Chinese Restaurant,Department Store,Bakery,Japanese Restaurant,Gourmet Shop,Gym
5,臺北市立南門國民中學 Taipei Municipal NanMen Junior High...,Middle School,1,Café,Noodle House,Hotel,Coffee Shop,History Museum,Chinese Restaurant,Metro Station,Supermarket,Historic Site,Convenience Store
6,臺北市立弘道國民中學 Taipei Municipal HongDao Junior Hig...,Middle School,1,Park,Food Court,Museum,Concert Hall,Bar,Fast Food Restaurant,Coffee Shop,Bistro,Sandwich Place,Bike Rental / Bike Share
7,臺北市立成功高級中學 Taipei Municipal ChengGong High School,High School,1,Dumpling Restaurant,Café,Noodle House,Chinese Restaurant,Hotel,Fast Food Restaurant,Thai Restaurant,Farmers Market,Coffee Shop,Historic Site
8,臺北市立福星國民小學 Taipei Municipal FuXing Elementary ...,Elementary School,1,Clothing Store,Hotel,Asian Restaurant,Noodle House,Bubble Tea Shop,Taiwanese Restaurant,Coffee Shop,Record Shop,Hotpot Restaurant,Historic Site
9,臺北市立建國高級中學 Taipei Municipal JianGuo High School,High School,1,Japanese Restaurant,History Museum,Noodle House,Garden,Museum,Convenience Store,Theater,Fast Food Restaurant,Chinese Breakfast Place,Buffet
10,臺北市立中正國民中學 Taipei Municipal ZhongZheng Junior ...,Middle School,1,Café,Breakfast Spot,Dumpling Restaurant,Noodle House,Bakery,Convenience Store,Korean Restaurant,Plaza,Coffee Shop,Chinese Breakfast Place
18,臺北市立長安國民中學 Taipei Municipal ChangAn Junior Hig...,Middle School,1,Japanese Restaurant,Café,Taiwanese Restaurant,Hotpot Restaurant,Hotel,Seafood Restaurant,Sushi Restaurant,Ramen Restaurant,Spa,Pastry Shop
19,臺北市立河堤國民小學 Taipei Municipal Heti Elementary Sc...,Elementary School,1,Café,Japanese Restaurant,Park,Chinese Restaurant,Convenience Store,Coffee Shop,Noodle House,Vegetarian / Vegan Restaurant,Taiwanese Restaurant,Hotpot Restaurant


### Cluster3 :

In [227]:
Taipei_school_merge.loc[Taipei_school_merge['Clusterlabel'] == 2, Taipei_school_merge.columns[[0,3] + list(range(4, Taipei_school_merge.shape[1]))]]

,School,Categories,Clusterlabel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,臺北市立民權國民中學 Taipei Municipal MingQuan Junior Hi...,Middle School,2,Taiwanese Restaurant,Convenience Store,Coffee Shop,Hotel,Bakery,Noodle House,Night Market,Gym / Fitness Center,Tea Room,Bus Station
13,臺北市立大橋國民小學 Taipei Municipal Dachiao Elementary...,Elementary School,2,Taiwanese Restaurant,Convenience Store,Noodle House,Food Truck,Bus Station,Dessert Shop,Snack Place,Night Market,Grocery Store,Tea Room
36,私立南山高級中學（附設國中部） Nanshan Private School,High School,2,Breakfast Spot,Arcade,Japanese Restaurant,Taiwanese Restaurant,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court
